# Use Case: Bulk Readability Analysis of Data Dump Filings

## Purpose

This is a powerful workflow that demonstrates the value of the FinancialReports data dump. It answers the question: **"Now that I've loaded my metadata into a database, what can I do with it?"**

This notebook will:
1. Connect to the SQLite database created by the `load_to_sqlite.py` script.
2. Query for a specific subset of filings (e.g., all annual reports).
3. For each filing, read its corresponding local markdown file.
4. Calculate its Gunning Fog readability score using our reusable analysis function.
5. Display the results in a sorted table and visualize the distribution of scores.

## 1. Setup

We import our required libraries. This includes `pandas` and `sqlalchemy` for data handling, and crucially, our `calculate_gunning_fog` function from the `/analysis` directory. This demonstrates how the cookbook's components can be combined.

We also import `matplotlib` for plotting.

In [ ]:
import pandas as pd
import sqlite3
import sys
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# --- Import from another directory ---
# To make the `utils.py` from our other example importable, we temporarily add its directory to Python's path.
gunning_fog_util_path = Path('../../analysis/calculate_gunning_fog/')
sys.path.append(str(gunning_fog_util_path.resolve()))

try:
    from utils import calculate_gunning_fog
    print("Successfully imported calculate_gunning_fog function.")
except ImportError:
    print("ERROR: Could not import 'calculate_gunning_fog'.")
    print(f"Please ensure '{gunning_fog_util_path / 'utils.py'}' exists.")

## 2. Connect and Query the Database

We'll connect to the `financialreports.db` file (assuming it's in the project root) and run a SQL query to get the filenames of all annual reports (`10-K`, `20-F`).

In [ ]:
DB_PATH = Path('../../financialreports.db')
DATA_DUMP_MARKDOWN_PATH = Path('/path/to/your/markdown/files/') # <-- IMPORTANT: User must change this path

if not DB_PATH.exists():
    print(f"ERROR: Database not found at '{DB_PATH}'.")
    print("Please run the 'load_metadata_csv_to_sqlite.py' script first.")
else:
    conn = sqlite3.connect(DB_PATH)
    
    # Query the database for all annual reports
    query = """
        SELECT markdown_filename 
        FROM filings_metadata 
        WHERE filing_type_code = '10-K' OR filing_type_code = '20-F'
    """
    
    df_filings = pd.read_sql_query(query, conn)
    conn.close()
    
    print(f"Found {len(df_filings)} annual report filings to analyze.")
    display(df_filings.head())

## 3. Process Filings and Calculate Scores

Now we loop through each filename, read the content of the corresponding markdown file, and calculate its Gunning Fog score. We'll use `tqdm` to show a progress bar, which is helpful for large datasets.

**Note:** This cell simulates the process. Since we don't have the actual data dump, it looks for the file and falls back to a sample text if it's not found. A real user would replace `DATA_DUMP_MARKDOWN_PATH` with the path to their files.

In [ ]:
results = []

# Use the sample text as a fallback if the real markdown file isn't found
fallback_text_path = gunning_fog_util_path / 'sample_text.txt'
with open(fallback_text_path, 'r') as f:
    fallback_text = f.read()

for filename in tqdm(df_filings['markdown_filename'], desc="Analyzing Filings"):
    file_path = DATA_DUMP_MARKDOWN_PATH / filename
    text_content = ''
    
    try:
        # In a real scenario, this is where you'd read the actual file
        if file_path.exists():
             with open(file_path, 'r', encoding='utf-8') as f:
                text_content = f.read()
        else:
            # For this example, we use fallback text if the file doesn't exist
            text_content = fallback_text
            
        # Calculate the score
        score = calculate_gunning_fog(text_content)
        results.append({'filename': filename, 'gunning_fog_score': score})

    except Exception as e:
        print(f"Could not process {filename}: {e}")
        results.append({'filename': filename, 'gunning_fog_score': None})


## 4. Display and Visualize Results

Finally, we'll load the results into a new DataFrame, sort it to find the least readable filings, and plot a histogram to see the overall distribution of scores.

In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='gunning_fog_score', ascending=False).reset_index(drop=True)

print("Top 5 Most Complex Filings (Highest Gunning Fog Score):")
display(df_results.head())

print("\n--- Score Distribution ---")
# Plotting the distribution
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 5))
df_results['gunning_fog_score'].hist(bins=20, ax=ax, color='#007acc', alpha=0.7)
ax.set_title('Distribution of Gunning Fog Scores for Annual Reports')
ax.set_xlabel('Gunning Fog Index')
ax.set_ylabel('Number of Filings')
plt.show()